##**Assignment 4**

##Задание 1

In [4]:
%%writefile sum_global.cu

#include <cuda_runtime.h>        // Подключаем CUDA Runtime API для работы с GPU
#include <iostream>              // Подключаем библиотеку для ввода и вывода
#include <vector>                // Подключаем контейнер std::vector
#include <chrono>                // Подключаем таймер для измерения времени на CPU
#include <cmath>                 // Подключаем математические функции (abs)

using namespace std;             // Используем пространство имён std, чтобы не писать std::

// Макрос для проверки ошибок CUDA-вызовов
#define CHECK_CUDA(call) do {                               \
    cudaError_t err = call;                                 \
    if (err != cudaSuccess) {                                \
        cerr << "CUDA error: "                              \
             << cudaGetErrorString(err)                      \
             << " at line " << __LINE__ << endl;           \
        exit(EXIT_FAILURE);                                 \
    }                                                       \
} while(0)

// CUDA-ядро для суммирования элементов массива с использованием глобальной памяти
__global__
void sum_global_kernel(const float* d_in, float* d_out, int n)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x; // Вычисляем глобальный индекс потока

    if (idx < n)                                     // Проверяем, не вышли ли за границы массива
        atomicAdd(d_out, d_in[idx]);                 // Атомарно добавляем элемент в общую сумму
}

// Последовательная функция для вычисления суммы на CPU
float cpu_sum(const vector<float>& data)
{
    float sum = 0.0f;                                // Переменная для хранения суммы

    for (float x : data)                             // Проходим по всем элементам массива
        sum += x;                                   // Последовательно суммируем элементы

    return sum;                                     // Возвращаем итоговую сумму
}

int main()
{
    const int N = 100000;                            // Размер массива

    vector<float> h_data(N);                         // Создаём массив на CPU

    for (int i = 0; i < N; i++)                      // Заполняем массив случайными числами
        h_data[i] = static_cast<float>(rand()) / RAND_MAX;

    // CPU вычисления

    auto cpu_start = chrono::high_resolution_clock::now(); // Запускаем таймер CPU
    float cpu_result = cpu_sum(h_data);                     // Вычисляем сумму на CPU
    auto cpu_end = chrono::high_resolution_clock::now();   // Останавливаем таймер CPU

    double cpu_time = chrono::duration<double, milli>(cpu_end - cpu_start).count(); // Время CPU

    // GPU вычисления

    float* d_in = nullptr;                           // Указатель на входной массив на GPU
    float* d_out = nullptr;                          // Указатель на результат на GPU

    CHECK_CUDA(cudaMalloc(&d_in, N * sizeof(float))); // Выделяем память под массив на GPU
    CHECK_CUDA(cudaMalloc(&d_out, sizeof(float)));    // Выделяем память под результат

    CHECK_CUDA(cudaMemcpy(d_in, h_data.data(),        // Копируем данные с CPU на GPU
                           N * sizeof(float),
                           cudaMemcpyHostToDevice));

    CHECK_CUDA(cudaMemset(d_out, 0, sizeof(float)));  // Обнуляем результат на GPU

    int threads = 256;                               // Количество потоков в одном блоке
    int blocks = (N + threads - 1) / threads;        // Вычисляем количество блоков

    auto gpu_start = chrono::high_resolution_clock::now(); // Запускаем таймер GPU
    sum_global_kernel<<<blocks, threads>>>(d_in, d_out, N); // Запускаем CUDA-ядро
    CHECK_CUDA(cudaDeviceSynchronize());              // Ждём завершения выполнения ядра
    auto gpu_end = chrono::high_resolution_clock::now();   // Останавливаем таймер GPU

    double gpu_time = chrono::duration<double, milli>(gpu_end - gpu_start).count(); // Время GPU

    float gpu_result = 0.0f;                          // Переменная для хранения результата GPU

    CHECK_CUDA(cudaMemcpy(&gpu_result, d_out,         // Копируем результат с GPU на CPU
                           sizeof(float),
                           cudaMemcpyDeviceToHost));

    // Вывод результатов

    cout << "CPU sum: " << cpu_result << endl;        // Вывод суммы, вычисленной на CPU
    cout << "GPU sum: " << gpu_result << endl;        // Вывод суммы, вычисленной на GPU
    cout << "Absolute error: "                         // Вывод абсолютной ошибки
         << abs(cpu_result - gpu_result) << endl;
    cout << "CPU time (ms): " << cpu_time << endl;    // Вывод времени CPU
    cout << "GPU time (ms): " << gpu_time << endl;    // Вывод времени GPU

    cudaFree(d_in);                                   // Освобождаем память GPU для входных данных
    cudaFree(d_out);                                  // Освобождаем память GPU для результата

    return 0;                                         // Завершаем программу
}

Overwriting sum_global.cu


In [5]:
!nvcc -arch=sm_75 -O2 sum_global.cu -o sum_global
!./sum_global

CPU sum: 49986.7
GPU sum: 49986.9
Absolute error: 0.191406
CPU time (ms): 0.302751
GPU time (ms): 0.465655


## **Вывод**

В рамках задания была реализована CUDA-программа для вычисления суммы элементов массива размером 100 000 с использованием глобальной памяти. Корректность реализации была подтверждена сравнением результатов, полученных на CPU и GPU.

Сумма, вычисленная на CPU, составила 49986.7, в то время как результат GPU составил 49986.9. Абсолютная погрешность равна 0.191406. Данная разница обусловлена использованием чисел с плавающей точкой и различным порядком выполнения операций сложения при последовательных вычислениях на CPU и параллельных вычислениях на GPU. Полученная погрешность является допустимой и не влияет на корректность результата.

Время выполнения последовательной реализации на CPU составило 0.30 мс, тогда как время выполнения CUDA-реализации на GPU составило 0.47 мс. Более длительное время выполнения на GPU объясняется накладными расходами на запуск CUDA-ядра и использованием атомарных операций в глобальной памяти, которые приводят к сериализации доступа потоков.

Таким образом, реализованная версия корректно вычисляет сумму элементов массива, однако не является оптимальной с точки зрения производительности. Данная реализация служит базовым вариантом для последующего сравнения с оптимизированными реализациями, использующими более эффективные типы памяти CUDA.

## Задание 2

In [12]:
%%writefile scan_shared.cu
#include <cuda_runtime.h>          // Подключаем CUDA Runtime API для работы с GPU
#include <iostream>                // Подключаем библиотеку для ввода и вывода
#include <vector>                  // Подключаем контейнер std::vector
#include <chrono>                  // Подключаем таймер для измерения времени
#include <cmath>                   // Подключаем математические функции (abs)

using namespace std;               // Используем пространство имён std

// Макрос для проверки ошибок CUDA
#define CHECK_CUDA(call) do {                               \
    cudaError_t err = call;                                 \
    if (err != cudaSuccess) {                               \
        cerr << "CUDA error: "                              \
             << cudaGetErrorString(err)                     \
             << " at line " << __LINE__ << endl;            \
        exit(EXIT_FAILURE);                                 \
    }                                                       \
} while(0)


// GPU KERNEL 1
// Префиксная сумма внутри каждого блока с использованием shared memory
__global__
void block_scan_kernel(const float* d_in,
                       float* d_out,
                       float* d_block_sums,
                       int n)
{
    extern __shared__ float sdata[];        // Разделяемая память блока

    int tid = threadIdx.x;                  // Индекс потока внутри блока
    int idx = blockIdx.x * blockDim.x + tid;// Глобальный индекс элемента

    // Загружаем элементы из глобальной памяти в shared memory
    if (idx < n)
        sdata[tid] = d_in[idx];             // Копируем элемент массива
    else
        sdata[tid] = 0.0f;                  // Если вышли за границы, записываем 0

    __syncthreads();                        // Синхронизация всех потоков блока

    // Алгоритм Hillis–Steele для вычисления префиксной суммы
    for (int offset = 1; offset < blockDim.x; offset <<= 1) {
        float temp = 0.0f;                  // Временная переменная
        if (tid >= offset)
            temp = sdata[tid - offset];     // Читаем значение на расстоянии offset
        __syncthreads();                    // Синхронизация перед обновлением
        sdata[tid] += temp;                 // Добавляем значение к текущему элементу
        __syncthreads();                    // Синхронизация после обновления
    }

    // Записываем результат префиксной суммы в глобальную память
    if (idx < n)
        d_out[idx] = sdata[tid];

    // Последний поток блока сохраняет сумму всего блока
    if (tid == blockDim.x - 1)
        d_block_sums[blockIdx.x] = sdata[tid];
}

// GPU KERNEL 2
// Добавление оффсетов блоков ко всем элементам
__global__
void add_offsets_kernel(float* d_data,
                        const float* d_offsets,
                        int n)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x; // Глобальный индекс элемента
    if (idx < n)
        d_data[idx] += d_offsets[blockIdx.x];        // Добавляем оффсет блока
}

// CPU PREFIX SUM
// Последовательная реализация префиксной суммы на CPU
void cpu_scan(const vector<float>& in, vector<float>& out)
{
    out.resize(in.size());               // Приводим размер выходного массива
    float acc = 0.0f;                    // Накопительная сумма
    for (size_t i = 0; i < in.size(); i++) {
        acc += in[i];                    // Добавляем текущий элемент
        out[i] = acc;                    // Записываем результат
    }
}

int main()
{
    const int N = 1'000'000;             // Размер массива
    const int threads = 256;             // Количество потоков в блоке
    int blocks = (N + threads - 1) / threads; // Количество блоков

    vector<float> h_in(N);               // Входной массив на CPU
    for (int i = 0; i < N; i++)           // Заполняем массив случайными числами
        h_in[i] = static_cast<float>(rand()) / RAND_MAX;

    // CPU

    vector<float> h_cpu;                 // Результат CPU
    auto cpu_start = chrono::high_resolution_clock::now(); // Запуск таймера CPU
    cpu_scan(h_in, h_cpu);               // Вычисление префиксной суммы на CPU
    auto cpu_end = chrono::high_resolution_clock::now();   // Остановка таймера CPU
    double cpu_time = chrono::duration<double, milli>(cpu_end - cpu_start).count();

    // GPU

    float *d_in, *d_out, *d_block_sums;  // Указатели на данные на GPU

    CHECK_CUDA(cudaMalloc(&d_in, N * sizeof(float)));        // Выделение памяти под вход
    CHECK_CUDA(cudaMalloc(&d_out, N * sizeof(float)));       // Память под результат
    CHECK_CUDA(cudaMalloc(&d_block_sums, blocks * sizeof(float))); // Память под суммы блоков

    CHECK_CUDA(cudaMemcpy(d_in, h_in.data(),                 // Копируем данные на GPU
                           N * sizeof(float),
                           cudaMemcpyHostToDevice));

    auto gpu_start = chrono::high_resolution_clock::now();   // Запуск таймера GPU

    block_scan_kernel<<<blocks, threads, threads * sizeof(float)>>>(
        d_in, d_out, d_block_sums, N);                        // Запуск первого ядра
    CHECK_CUDA(cudaDeviceSynchronize());                      // Ожидание завершения

    // Копируем суммы блоков на CPU
    vector<float> h_block_sums(blocks);
    vector<float> h_offsets(blocks);

    CHECK_CUDA(cudaMemcpy(h_block_sums.data(),
                           d_block_sums,
                           blocks * sizeof(float),
                           cudaMemcpyDeviceToHost));

    // Вычисляем оффсеты блоков на CPU
    float running = 0.0f;
    for (int i = 0; i < blocks; i++) {
        h_offsets[i] = running;        // Оффсет текущего блока
        running += h_block_sums[i];    // Обновляем накопленную сумму
    }

    float* d_offsets;                  // Указатель на оффсеты на GPU
    CHECK_CUDA(cudaMalloc(&d_offsets, blocks * sizeof(float)));
    CHECK_CUDA(cudaMemcpy(d_offsets,
                           h_offsets.data(),
                           blocks * sizeof(float),
                           cudaMemcpyHostToDevice));

    add_offsets_kernel<<<blocks, threads>>>(d_out, d_offsets, N); // Добавление оффсетов
    CHECK_CUDA(cudaDeviceSynchronize());                           // Ожидание

    auto gpu_end = chrono::high_resolution_clock::now();          // Остановка таймера GPU
    double gpu_time = chrono::duration<double, milli>(gpu_end - gpu_start).count();

    // Проверка

    vector<float> h_gpu(N);                 // Результат GPU на CPU
    CHECK_CUDA(cudaMemcpy(h_gpu.data(),
                           d_out,
                           N * sizeof(float),
                           cudaMemcpyDeviceToHost));

    float max_error = 0.0f;                 // Максимальная ошибка
    for (int i = 0; i < N; i++)
        max_error = max(max_error, abs(h_cpu[i] - h_gpu[i]));

    // Вывод

    cout << "Array size: " << N << endl;
    cout << "Threads per block: " << threads << endl << endl;

    cout << "CPU time (ms): " << cpu_time << endl;
    cout << "GPU time (ms): " << gpu_time << endl;
    cout << "Speedup (CPU / GPU): " << cpu_time / gpu_time << "x" << endl << endl;

    cout << "First 10 elements:" << endl;
    cout << "CPU: ";
    for (int i = 0; i < 10; i++) cout << h_cpu[i] << " ";
    cout << endl;

    cout << "GPU: ";
    for (int i = 0; i < 10; i++) cout << h_gpu[i] << " ";
    cout << endl << endl;

    cout << "Last element (total sum):" << endl;
    cout << "CPU: " << h_cpu[N - 1] << endl;
    cout << "GPU: " << h_gpu[N - 1] << endl << endl;

    cout << "Max absolute error: " << max_error << endl;

    cudaFree(d_in);                         // Освобождаем память GPU
    cudaFree(d_out);
    cudaFree(d_block_sums);
    cudaFree(d_offsets);

    return 0;                               // Завершаем программу
}

Overwriting scan_shared.cu


In [13]:
!nvcc -arch=sm_75 -O2 scan_shared.cu -o scan_shared
!./scan_shared

Array size: 1000000
Threads per block: 256

CPU time (ms): 3.63966
GPU time (ms): 0.461058
Speedup (CPU / GPU): 7.89414x

First 10 elements:
CPU: 0.840188 1.23457 2.01767 2.81611 3.72776 3.92531 4.26053 5.02876 5.30654 5.86051 
GPU: 0.840188 1.23457 2.01767 2.81611 3.72776 3.92531 4.26053 5.02876 5.30654 5.86051 

Last element (total sum):
CPU: 500004
GPU: 500007

Max absolute error: 5.53125


## **Вывод**

В рамках задания была реализована CUDA-программа для вычисления префиксной суммы массива размером 1 000 000 элементов с использованием разделяемой памяти. Корректность реализации была проверена путём сравнения результатов, полученных на CPU и GPU.

Время выполнения последовательной реализации на CPU составило 3.64 мс, тогда как время выполнения реализации на GPU составило 0.46 мс. Таким образом, использование GPU позволило получить ускорение примерно в 7.9 раза по сравнению с CPU. Это показывает эффективность параллельных вычислений и применения разделяемой памяти при обработке больших массивов данных.

Сравнение первых элементов префиксной суммы показало полное совпадение результатов CPU и GPU, что подтверждает корректность вычислений. Значение последнего элемента массива, соответствующее полной сумме, отличается на небольшую величину. Максимальная абсолютная погрешность составила 5.53, что объясняется использованием чисел с плавающей точкой и различием порядка операций сложения при параллельных вычислениях на GPU. Данная погрешность является допустимой для типа данных float.

Таким образом, реализованная версия префиксной суммы с использованием разделяемой памяти корректно выполняет поставленную задачу и существенно превосходит последовательную реализацию на CPU по производительности. Результаты эксперимента подтверждают, что оптимизация доступа к памяти и использование архитектуры GPU позволяют значительно ускорить вычисление операций сканирования.

## Задание 3

In [18]:
%%writefile hybrid_sum.cu

#include <cuda_runtime.h>          // Подключаем CUDA Runtime API для работы с GPU
#include <iostream>                // Подключаем библиотеку для ввода и вывода
#include <vector>                  // Подключаем контейнер std::vector
#include <chrono>                  // Подключаем таймер для измерения времени
#include <cmath>                   // Подключаем математические функции (abs)
#include <thread>                  // std::thread

using namespace std;               // Используем пространство имён std

// Макрос для проверки ошибок CUDA
#define CHECK_CUDA(call) do {                               \
    cudaError_t err = call;                                 \
    if (err != cudaSuccess) {                               \
        cerr << "CUDA error: "                              \
             << cudaGetErrorString(err)                     \
             << " at line " << __LINE__ << endl;            \
        exit(EXIT_FAILURE);                                 \
    }                                                       \
} while(0)

//  GPU KERNEL
// CUDA-ядро для суммирования элементов массива с использованием глобальной памяти
__global__
void sum_kernel(const float* d_in, float* d_out, int n)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x; // Вычисляем глобальный индекс потока

    if (idx < n)                                     // Проверяем выход за границы массива
        atomicAdd(d_out, d_in[idx]);                 // Атомарно добавляем элемент к общей сумме
}

//  CPU SUM
// Последовательная функция для суммирования части массива на CPU
float cpu_sum_part(const vector<float>& data, int start, int end)
{
    float sum = 0.0f;                                // Переменная для накопления суммы

    for (int i = start; i < end; i++)                // Проходим по заданному диапазону
        sum += data[i];                              // Добавляем текущий элемент

    return sum;                                      // Возвращаем частичную сумму
}

int main()
{
    const int N = 1'000'000;                         // Размер массива
    const int threads = 256;                         // Количество потоков в одном блоке

    vector<float> h_data(N);                         // Создаём массив на CPU

    for (int i = 0; i < N; i++)                      // Заполняем массив случайными числами
        h_data[i] = static_cast<float>(rand()) / RAND_MAX;

    //  CPU ONLY

    auto cpu_start = chrono::high_resolution_clock::now(); // Запускаем таймер CPU
    float cpu_result = cpu_sum_part(h_data, 0, N);         // Считаем сумму всего массива на CPU
    auto cpu_end = chrono::high_resolution_clock::now();   // Останавливаем таймер CPU

    double cpu_time =
        chrono::duration<double, milli>(cpu_end - cpu_start).count(); // Время CPU

    //  GPU ONLY

    float *d_in, *d_out;                            // Указатели на данные на GPU

    CHECK_CUDA(cudaMalloc(&d_in, N * sizeof(float))); // Выделяем память под массив на GPU
    CHECK_CUDA(cudaMalloc(&d_out, sizeof(float)));    // Выделяем память под результат

    CHECK_CUDA(cudaMemcpy(d_in, h_data.data(),       // Копируем массив с CPU на GPU
                           N * sizeof(float),
                           cudaMemcpyHostToDevice));

    CHECK_CUDA(cudaMemset(d_out, 0, sizeof(float))); // Обнуляем результат на GPU

    int blocks = (N + threads - 1) / threads;        // Вычисляем количество блоков

    auto gpu_start = chrono::high_resolution_clock::now(); // Запускаем таймер GPU
    sum_kernel<<<blocks, threads>>>(d_in, d_out, N); // Запускаем CUDA-ядро
    CHECK_CUDA(cudaDeviceSynchronize());              // Ждём завершения GPU
    auto gpu_end = chrono::high_resolution_clock::now();   // Останавливаем таймер GPU

    double gpu_time =
        chrono::duration<double, milli>(gpu_end - gpu_start).count(); // Время GPU

    float gpu_result = 0.0f;                          // Переменная для GPU-результата
    CHECK_CUDA(cudaMemcpy(&gpu_result, d_out,         // Копируем результат с GPU на CPU
                           sizeof(float),
                           cudaMemcpyDeviceToHost));

    // HYBRID CPU + GPU

    int mid = N / 2;                                 // Делим массив пополам

    CHECK_CUDA(cudaMemset(d_out, 0, sizeof(float))); // Обнуляем GPU-результат

    auto hybrid_start = chrono::high_resolution_clock::now(); // Запуск таймера гибридной версии

    float cpu_partial = 0.0f;                        // Частичная сумма CPU

    // Запускаем вычисление первой половины массива на CPU в отдельном потоке
    thread cpu_thread([&]() {
        cpu_partial = cpu_sum_part(h_data, 0, mid);
    });

    // GPU вычисляет сумму второй половины массива параллельно с CPU
    sum_kernel<<<(N - mid + threads - 1) / threads, threads>>>(
        d_in + mid, d_out, N - mid);

    cpu_thread.join();                               // Ждём завершения CPU-потока
    CHECK_CUDA(cudaDeviceSynchronize());             // Ждём завершения GPU

    auto hybrid_end = chrono::high_resolution_clock::now(); // Остановка таймера

    double hybrid_time =
        chrono::duration<double, milli>(hybrid_end - hybrid_start).count(); // Время гибридной версии

    float gpu_partial = 0.0f;                        // Частичная сумма GPU
    CHECK_CUDA(cudaMemcpy(&gpu_partial, d_out,       // Копируем частичный результат GPU
                           sizeof(float),
                           cudaMemcpyDeviceToHost));

    float hybrid_result = cpu_partial + gpu_partial; // Итоговая сумма (CPU + GPU)

    // Вывод

    cout << "Array size: " << N << endl << endl;

    cout << "CPU only:" << endl;
    cout << "Result: " << cpu_result << endl;
    cout << "Time (ms): " << cpu_time << endl << endl;

    cout << "GPU only:" << endl;
    cout << "Result: " << gpu_result << endl;
    cout << "Time (ms): " << gpu_time << endl << endl;

    cout << "Hybrid CPU + GPU:" << endl;
    cout << "Result: " << hybrid_result << endl;
    cout << "Time (ms): " << hybrid_time << endl << endl;

    cout << "Speedup (CPU / Hybrid): "
         << cpu_time / hybrid_time << "x" << endl;

    cudaFree(d_in);                                  // Освобождаем память GPU
    cudaFree(d_out);

    return 0;                                        // Завершаем программу
}

Overwriting hybrid_sum.cu


In [19]:
!nvcc -arch=sm_75 -O2 hybrid_sum.cu -o hybrid_sum
!./hybrid_sum

Array size: 1000000

CPU only:
Result: 500004
Time (ms): 2.56455

GPU only:
Result: 500003
Time (ms): 3.55758

Hybrid CPU + GPU:
Result: 500008
Time (ms): 1.80905

Speedup (CPU / Hybrid): 1.41762x


## **Вывод**

В рамках задания была реализована гибридная программа, в которой обработка массива выполнялась параллельно на CPU и GPU. Массив размером 1 000 000 элементов был разделён на две части: первая часть обрабатывалась на CPU, а вторая на GPU. После завершения вычислений частичные результаты объединялись на CPU.

Последовательная реализация на CPU показала время выполнения 2.56 мс. Реализация, использующая только GPU, выполнила вычисления за 3.56 мс, что оказалось медленнее CPU. Это связано с накладными расходами на запуск CUDA-ядра и использованием атомарных операций в глобальной памяти, которые приводят к сериализации доступа потоков.

Гибридная реализация продемонстрировала наилучший результат по времени выполнения 1.81 мс. Это обеспечило ускорение примерно в 1.42 раза по сравнению с последовательной реализацией на CPU. Полученные результаты показывают, что одновременное использование CPU и GPU позволяет более эффективно задействовать вычислительные ресурсы системы.

Результаты вычислений для CPU-, GPU- и гибридной реализаций совпадают с допустимой погрешностью, обусловленной вычислениями с плавающей точкой и различием порядка операций сложения. Это подтверждает корректность реализованного гибридного подхода.

Таким образом, эксперимент подтвердил, что гибридная модель вычислений может быть более эффективной, чем использование только CPU или только GPU, и демонстрирует преимущества гетерогенных вычислений при обработке больших массивов данных.

## Задание 4

In [20]:
%%writefile mpi_sum.cpp

#include <mpi.h>          // Основная библиотека MPI
#include <iostream>      // Ввод и вывод
#include <vector>        // Контейнер std::vector
#include <cstdlib>       // rand, RAND_MAX

using namespace std;     // Используем пространство имён std

int main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);              // Инициализация MPI

    int rank, size;                      // rank - номер процесса, size - их количество
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    const int N = 1'000'000;             // Общий размер массива
    int local_n = N / size;              // Размер части для каждого процесса

    vector<float> local_data(local_n);   // Локальный массив процесса

    vector<float> data;                  // Полный массив (только у процесса 0)
    if (rank == 0) {
        data.resize(N);                  // Выделяем память под массив
        for (int i = 0; i < N; i++)       // Заполняем массив случайными числами
            data[i] = static_cast<float>(rand()) / RAND_MAX;
    }

    MPI_Barrier(MPI_COMM_WORLD);          // Синхронизация процессов
    double start_time = MPI_Wtime();      // Начало замера времени

    MPI_Scatter(data.data(),              // Отправляем данные от процесса 0
                local_n, MPI_FLOAT,       // Размер и тип части
                local_data.data(),        // Локальный буфер
                local_n, MPI_FLOAT,       // Размер и тип принимаемых данных
                0, MPI_COMM_WORLD);       // Корневой процесс — 0

    float local_sum = 0.0f;               // Локальная сумма
    for (int i = 0; i < local_n; i++)
        local_sum += local_data[i];       // Суммируем локальную часть массива

    float global_sum = 0.0f;              // Итоговая сумма

    MPI_Reduce(&local_sum,                // Локальные суммы
               &global_sum,               // Итоговый результат
               1, MPI_FLOAT,              // Один элемент типа float
               MPI_SUM,                   // Операция суммирования
               0, MPI_COMM_WORLD);        // Корневой процесс — 0

    double end_time = MPI_Wtime();        // Конец замера времени

    if (rank == 0) {
        cout << "Processes: " << size << endl;
        cout << "Result (sum): " << global_sum << endl;
        cout << "Execution time (s): "
             << end_time - start_time << endl << endl;
    }

    MPI_Finalize();                       // Завершение MPI
    return 0;
}

Writing mpi_sum.cpp


In [21]:
!mpic++ mpi_sum.cpp -o mpi_sum

In [28]:
!mpirun --allow-run-as-root --oversubscribe -np 2 ./mpi_sum
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_sum
!mpirun --allow-run-as-root --oversubscribe -np 8 ./mpi_sum

Processes: 2
Result (sum): 500008
Execution time (s): 0.00317955

Processes: 4
Result (sum): 500007
Execution time (s): 0.00443688

Processes: 8
Result (sum): 500007
Execution time (s): 0.00353687



## **Вывод**

В рамках задания была реализована распределённая программа с использованием технологии MPI для обработки массива данных размером 1 000 000 элементов. Массив был разделён между процессами, каждый процесс выполнял локальные вычисления, после чего частичные результаты объединялись с помощью операции MPI_Reduce.

Экспериментальные замеры времени выполнения были проведены для 2, 4 и 8 процессов. При использовании 2 процессов время выполнения составило 0.00318 с. При увеличении числа процессов до 4 время выполнения возросло до 0.00444 с, а при использовании 8 процессов составило 0.00354 с. Полученные результаты показывают, что увеличение числа процессов не всегда приводит к уменьшению времени выполнения.

Такое поведение объясняется накладными расходами на коммуникацию между процессами и синхронизацию данных. Для относительно небольшого объёма вычислений затраты на обмен данными могут превосходить выигрыш от дополнительного параллелизма, что приводит к снижению эффективности масштабирования.

Результаты вычислений для всех запусков совпадают с допустимой погрешностью, обусловленной использованием чисел с плавающей точкой. Это подтверждает корректность реализации распределённого алгоритма.

Таким образом, эксперимент показал, что MPI-подход эффективен для распределения вычислений, однако его производительность существенно зависит от соотношения объёма вычислений и коммуникационных затрат. Для достижения лучшей масштабируемости требуется увеличение размера задачи или более сложные вычисления внутри каждого процесса.